This file contains the code to differentiate the differences statistically.

13/11/2022

In [1]:
import numpy as np
import scipy
import scipy.stats as stats
import scikit_posthocs as sp
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
import seaborn as sns

In [2]:
os.chdir('D:\\4th Year\\Semester 7\\BI4313 Sem Project\\IN-comparative-analysis\\IN-comparative-analysis\\Quant\\Updated_values')

Assumptions for ANOVA - 
1. Measurements are made randomly from a population
2. Variable is normally distributed in each of the populations
3. Variance is the same in all k populations

BUT: ANOVA is robust to departures from the assumption of equal variance (Ch 15, Whitlock and Schuster), but only if samples are all large, about the same size and no more than 10-fold difference among variances.

Normality test for ANOVA - scipy.stats.shapiro(x)

Shapiro Wilk test is suited for smaller sample size < 50. If something is not normally distributed, then either the data has to be transformed, or we've to use a test that doesn't assume normality

In [38]:
#data = pd.read_csv("01. Song_quantifiers.csv")
#data = pd.read_csv("02. Syll_quantifiers.csv")
data = pd.read_csv("07. Start-Self Transition Prob.csv")
#data = pd.read_csv("08. IN Start-Self Transition Prob.csv")

#Subset data and keep the parameter
df1 = data[['Bird name', 'Species', 'Self-transition prob']] #c1

species = ['BCC', 'BF', 'JF', 'ZF']

for i in range(len(species)):
    df2 = df1.loc[df1['Species'] == species[i]]
    df3 = df2['Self-transition prob'] #c2
    k2, p = stats.shapiro(df3)
    alpha = 0.05
    if p < alpha:
        print(f'The values of this species are NOT normally distrbuted: {species[i]}')
    else:
        print(f'Normally distrbuted: {species[i]}')
    std_dev = np.std(df3)
    var = std_dev**2
    print(f'Variance = {var}')


The values of this species are NOT normally distrbuted: BCC
Variance = 0.03938128543132299
The values of this species are NOT normally distrbuted: BF
Variance = 0.02642834467120182
The values of this species are NOT normally distrbuted: JF
Variance = 0.08310828821324451
The values of this species are NOT normally distrbuted: ZF
Variance = 0.040044285982360706


ANOVA

In [39]:
a1 = df1.loc[df1['Species'] == 'BCC']
a2 = a1['Self-transition prob'].to_list()

b1 = df1.loc[df1['Species'] == 'BF']
b2 = b1['Self-transition prob'].to_list()

c1 = df1.loc[df1['Species'] == 'JF']
c2 = c1['Self-transition prob'].to_list()

d1 = df1.loc[df1['Species'] == 'ZF']
d2 = d1['Self-transition prob'].to_list()

In [ ]:
stats.f_oneway(a2, b2, c2, d2)

If ANOVA p-value is significant, it tells us one group is significantly different than the others, but it doesn't really tell us which one it is. So, we do a post hoc Tukey-Kramer test (Tukey for equal sample size, T-K for unequal), which allows us to make pairwise comparisons.

#90% sure that 'pairwise_tukeyhsd' is the same as Tukey-Kramer test, it's not explicitly mentioned in the documentation.

Alaternative - use Scheffe's method for a post hoc test - applicable in all conditions, but more conservative than Tukey

In [75]:
df = pd.DataFrame({
    'values' : a2 + b2 + c2 + d2,
    'label' : np.repeat(['BCC', 'BF', 'JF', 'ZF'], repeats=[len(a2), len(b2), len(c2), len(d2)])
    })

#Not valid right now, because variance is too much and sample size is unequal
tuk = pairwise_tukeyhsd(endog=df['values'], groups=df['label'], alpha=0.05)
print(tuk)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
   BCC     BF    8.648  0.001   5.7075 11.5885   True
   BCC     JF   3.5754 0.0134   0.6348  6.5159   True
   BCC     ZF    0.843  0.781   -1.736   3.422  False
    BF     JF  -5.0727 0.0015  -8.3349 -1.8104   True
    BF     ZF   -7.805  0.001 -10.7455 -4.8645   True
    JF     ZF  -2.7323 0.0749  -5.6729  0.2082  False
-----------------------------------------------------


Kruskal-Wallis test - non-parametric alternative for ANOVA (similar to Mann Whitney U test, but for multiple groups). But this test is less powerful for small sample sizes. Assumptions - 
1. All group saples were drawn randomly
2. Distribution of variable must have the same shape in every population

In [40]:
stats.kruskal(a2, b2, c2, d2)

KruskalResult(statistic=17.718210233145633, pvalue=0.0005028042641747661)

Dunn test - this is the non-parametric post-hoc test used to do pairwise comparison after Kruskal-Wallis test

In [41]:
data = [a2, b2, c2, d2]
col = ['BCC', 'BF', 'JF', 'ZF']

dunn = sp.posthoc_dunn(data)
# Doesn't give significant result, probably because of extra corrections?
#dunn = sp.posthoc_dunn(data, p_adjust='bonferroni') 

dunn.columns = col
dunn.index = col

print(dunn)
print(dunn < 0.05)

          BCC        BF        JF        ZF
BCC  1.000000  0.526943  0.000031  0.134983
BF   0.526943  1.000000  0.004837  0.576338
JF   0.000031  0.004837  1.000000  0.008733
ZF   0.134983  0.576338  0.008733  1.000000
       BCC     BF     JF     ZF
BCC  False  False   True  False
BF   False  False   True  False
JF    True   True  False   True
ZF   False  False   True  False


------------------------------------------------------------------------------------------------------------------

Signed Rank test for Amplitude, Frequency, Duration and Interval of first and fifth syllable

Wilcoxon test assumes even distribution around the median i.e., no skew which is similar to the normality assumption, restricting its use.
If normality condition is not met, simply Sign Test can be used.

The Wilcoxon signed-rank test tests the null hypothesis that two related paired samples come from the same distribution. In particular, it tests whether the distribution of the differences x - y is symmetric about zero. It is a non-parametric version of the paired T-test.

In [3]:
data = pd.read_csv("06. Avg internote interval values.csv") #c1

df1 = data[['Bird name', 'Species', '#1', '#5']]

species = ['BCC', 'BF', 'JF', 'ZF']

for i in range(len(species)):
    df2 = df1.loc[df1['Species'] == species[i]]
    df3 = df2['#1']
    k1, p1 = stats.shapiro(df3)
    df4 = df2['#5']
    k2, p2 = stats.shapiro(df4)
    alpha = 0.05
    if p1 < alpha:
        print(f'The values of this species #1 are NOT normally distrbuted: {species[i]}')
    else:
        print(f'Normally distrbuted #1: {species[i]}')
    if p2 < alpha:
        print(f'The values of this species #5 are NOT normally distrbuted: {species[i]}')
    else:
        print(f'Normally distrbuted #5: {species[i]}')
    std_dev1 = np.std(df3)
    std_dev2 = np.std(df4)
    var1 = std_dev1**2
    var2 = std_dev2**2
    print(f'Variance #1 = {var1} and Variance #5 = {var2}')


Normally distrbuted #1: BCC
Normally distrbuted #5: BCC
Variance #1 = 8441.236523775713 and Variance #5 = 1233.7611071954182
Normally distrbuted #1: BF
Normally distrbuted #5: BF
Variance #1 = 20948.311494024667 and Variance #5 = 732.9232362166167
Normally distrbuted #1: JF
Normally distrbuted #5: JF
Variance #1 = 18000.229682536414 and Variance #5 = 9645.689852715213
Normally distrbuted #1: ZF
Normally distrbuted #5: ZF
Variance #1 = 13470.565616813237 and Variance #5 = 291.04630716036627


In [4]:
species = ['BCC', 'BF', 'JF', 'ZF']

for i in range(len(species)):
    df2 = df1.loc[df1['Species'] == species[i]]
    v1 = df2['#1']
    v2 = df2['#5']
    #'alternative' states the alternative hypothesis - 
    #for amplitude and frequency, it's 'greater', but interval it's 'less'
    s, p = stats.wilcoxon(v2, v1, alternative='two-sided')
    print(f'Wilcoxon test p-value for {species[i]} is {p}')

Wilcoxon test p-value for BCC is 0.0078125
Wilcoxon test p-value for BF is 0.0625
Wilcoxon test p-value for JF is 0.0625
Wilcoxon test p-value for ZF is 0.0078125


In [53]:
def main(df, CI):
    
    alpha = 1-CI/100
    
    if len(df.index)<2: df = df.rename(columns = {"p-unc" : "pval"})    #the pval column  has different names based on test and numerosity
    else: df = df.rename(columns = {"p-corr" : "pval"})
    
    df.rename({'A': 'group1', 'B': 'group2', "pval":"p-adj"}, axis="columns", inplace=True)
    
    '''
    Creates a compact letter display. This creates a dataframe consisting of
    2 columns, a column containing the treatment groups and a column containing
    the letters that have been assigned to the treatment groups. These letters
    are part of what's called the compact letter display. Treatment groups that
    share at least 1 letter are similar to each other, while treatment groups
    that don't share any letters are significantly different from each other.
    Parameters
    ----------
    df : Pandas dataframe
        Pandas dataframe containing raw Tukey test results from statsmodels.
    alpha : float
        The alpha value. The default is 0.05.
    Returns
    -------
    A dataframe representing the compact letter display, created from the Tukey
    test results.
    '''
    df["p-adj"] = df["p-adj"].astype(float)

    # Creating a list of the different treatment groups from Tukey's
    group1 = set(df.group1.tolist())  # Dropping duplicates by creating a set
    group2 = set(df.group2.tolist())  # Dropping duplicates by creating a set
    groupSet = group1 | group2  # Set operation that creates a union of 2 sets
    groups = sorted(list(groupSet))

    # Creating lists of letters that will be assigned to treatment groups
    letters = list(string.ascii_lowercase+string.digits)[:len(groups)]
    cldgroups = letters

    # the following algoritm is a simplification of the classical cld,

    cld = pd.DataFrame(list(zip(groups, letters, cldgroups)))
    cld[3]=""
    
    for row in df.itertuples():
        if df["p-adj"][row[0]] > (alpha):
            cld.iat[groups.index(df["group1"][row[0]]), 2] += cld.iat[groups.index(df["group2"][row[0]]), 1]
            cld.iat[groups.index(df["group2"][row[0]]), 2] += cld.iat[groups.index(df["group1"][row[0]]), 1]
            
        if df["p-adj"][row[0]] < (alpha):
                cld.iat[groups.index(df["group1"][row[0]]), 3] +=  cld.iat[groups.index(df["group2"][row[0]]), 1]
                cld.iat[groups.index(df["group2"][row[0]]), 3] +=  cld.iat[groups.index(df["group1"][row[0]]), 1]

    cld[2] = cld[2].apply(lambda x: "".join(sorted(x)))
    cld[3] = cld[3].apply(lambda x: "".join(sorted(x)))
    cld.rename(columns={0: "groups"}, inplace=True)

    # this part will reassign the final name to the group
    # for sure there are more elegant ways of doing this
    cld["labels"] = ""
    letters = list(string.ascii_lowercase)
    unique = []
    for item in cld[2]:

        for fitem in cld["labels"].unique():
            for c in range(0, len(fitem)):
                if not set(unique).issuperset(set(fitem[c])):
                    unique.append(fitem[c])
        g = len(unique)

        for kitem in cld[1]:
            if kitem in item:
                #Checking if there are forbidden pairing (proposition of solution to the imperfect script)                
                forbidden = set()
                for row in cld.itertuples():
                    if letters[g] in row[5]:
                        forbidden |= set(row[4])
                if kitem in forbidden:
                    g=len(unique)+1
               
                if cld["labels"].loc[cld[1] == kitem].iloc[0] == "":
                   cld["labels"].loc[cld[1] == kitem] += letters[g] 
               
                # Checking if columns 1 & 2 of cld share at least 1 letter
                if len(set(cld["labels"].loc[cld[1] == kitem].iloc[0]).intersection(cld.loc[cld[2] == item, "labels"].iloc[0])) <= 0:
                    if letters[g] not in list(cld["labels"].loc[cld[1] == kitem].iloc[0]):
                        cld["labels"].loc[cld[1] == kitem] += letters[g]
                    if letters[g] not in list(cld["labels"].loc[cld[2] == item].iloc[0]):
                        cld["labels"].loc[cld[2] == item] += letters[g]

    cld = cld.sort_values("labels")
    cld.drop(columns=[1, 2, 3], inplace=True)
    print(cld)
    print('\n')
    print('\n')
    return(cld)